In [2]:
from IPython.display import Markdown, display
import PyPDF2
import requests
import os
from PyPDF2 import PdfReader
from openai import OpenAI
from dotenv import load_dotenv

In [3]:
def load_api_key():
    load_dotenv()
    api_key = os.getenv("OPENAI_API_KEY")

    # Check the key

    if not api_key:
        print(
            "No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!"
        )
    elif not api_key.startswith("sk-proj-"):
        print(
            "An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook"
        )
    elif api_key.strip() != api_key:
        print(
            "An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook"
        )
    else:
        print("API key found and looks good so far!")
    return api_key

In [4]:
api_key = load_api_key()
openai = OpenAI()

API key found and looks good so far!


In [18]:
def read_all_pdfs(folder_path: str = "data", include_subdir=True) -> str:
    # List to store extracted text from all PDFs
    all_text = []
    if include_subdir:
        # Walk through all folders and files
        for root, dirs, files in os.walk(folder_path):
            for file_name in files:
                if file_name.endswith(".pdf"):  # Check if the file is a PDF
                    file_path = os.path.join(root, file_name)  # Full file path
                    
                    # Open and read the PDF
                    with open(file_path, 'rb') as file:
                        reader = PdfReader(file)
                        text = ""
                        
                        # Extract text from each page
                        for page in reader.pages:
                            text += page.extract_text()
                        
                        # Append extracted text to the list
                        all_text.append(f"--- Text from {file_name} in {root} ---\n{text}\n")
                        print(f"Processed: {file_name} from {root}")

    else:

        # Loop through all files in the folder
        for file_name in os.listdir(folder_path):
            if file_name.endswith(".pdf"):  # Check if the file is a PDF
                file_path = os.path.join(folder_path, file_name)  # Get full file path

                # Open and read the PDF
                with open(file_path, "rb") as file:
                    reader = PdfReader(file)
                    text = ""
                    for page in reader.pages:
                        text += page.extract_text()  # Extract text from each page

                    # Append extracted text to the list
                    all_text.append(f"--- Text from {file_name} ---\n{text}\n")
                    print(f"Processed: {file_name}")

    final_text = "\n".join(all_text)

    return final_text

In [28]:
final_text = read_all_pdfs("data/reflexive")

Processed: https___cloudfront.baselang.com_media_lessons_77.-_Reflexives_da4CzkB.pdf from data/reflexive


In [24]:
OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "llama3.2"


def chat_with_ollama(prompt):
    messages = [{"role": "user", "content": prompt}]
    payload = {"model": MODEL, "messages": messages, "stream": False}
    response = requests.post(OLLAMA_API, json=payload, headers=HEADERS)
    return response.json()["message"]["content"]

In [25]:
def chat_with_gpt(message, system_prompt=None):
    response = openai.chat.completions.create(
        model="gpt-4o-mini", messages=[{"role": "user", "content": message}]
    )
    return response.choices[0].message.content

In [29]:
# Create a messages list using the same format that we used for OpenAI
user_prompt = "This is a teaching presentation for spanish verbs. Please find all the new spanish \
    verbs and return them with their Engligh meaning. Here is the content of the slides. Only give me the list of verbs and their translations."
# user_prompt += read_pdf('data\https___cloudfront.baselang.com_media_lessons_130.-_Verbs_Level_5_-_Part_2.pdf')
user_prompt += final_text

In [30]:
verbs_list = chat_with_gpt(user_prompt)
display(Markdown(verbs_list))

1. Despertarse - To wake up
2. Peinarse - To brush the hair
3. Cepillarse - To brush
4. Levantarse - To get up, to stand up
5. Vestirse - To get dressed
6. Lavarse - To wash
7. Bañarse - To bathe
8. Maquillarse - To put makeup on
9. Enfermarse - To get sick
10. Mudarse - To move
11. Divertirse - To have fun
12. Sentirse - To feel
13. Alegrarse - To cheer up
14. Acostumbrarse - To get used to
15. Casarse - To get married
16. Graduarse - To graduate
17. Ducharse - To shower
18. Afeitarse - To shave
19. Enojarse - To get angry
20. Olvidarse - To forget

In [41]:
user_prompt = f"""
Please conjugate the following Spanish verbs in a **single tabular format** for the tenses: Present, Preterite, Imperfect, Conditional, and Future. Follow these exact rules:

1. **Table Structure**:
   - Use the following **columns**:
     - Verb *(English)*
     - Pronoun *(Yo, Tú, Él/Ella/Usted, Nosotros, Ellos/Ellas/Ustedes)*
     - Present Tense
     - Preterite
     - Imperfect
     - Conditional
     - Future
   - The conjugations for all verbs should appear in the same table.

2. **Verb Section**:
   - For each verb, include its **Spanish name** with the **English translation** (e.g., "**Empezar** *(to start)*").
   - The verb name should only appear once at the top of its section.

3. **Clean Formatting**:
   - Each cell under tense columns should contain **only one conjugated form** for the corresponding pronoun.
   - Ensure the output avoids duplicate conjugations or unnecessary text.

4. **Colombian Spanish**:
   - Use Colombian Spanish conjugations (do not include "vosotros").

5. **Single Table**:
   - Combine all conjugations into **one single table**.
   - Separate each verb section clearly for better readability.

{verbs_list}

The final output should look like this:

| Verb *(English)*     | Pronoun            | Present     | Preterite    | Imperfect    | Conditional    | Future       |
|-----------------------|--------------------|-------------|-------------|-------------|----------------|--------------|
| **Empezar (to start)**| Yo                 | Empiezo     | Empecé      | Empezaba    | Empezaría      | Empezaré     |
|                       | Tú                 | Empiezas    | Empezaste   | Empezabas   | Empezarías     | Empezarás    |
|                       | Él/Ella/Usted      | Empieza     | Empezó      | Empezaba    | Empezaría      | Empezará     |
|                       | Nosotros           | Empezamos   | Empezamos   | Empezábamos | Empezaríamos   | Empezaremos  |
|                       | Ellos/Ellas/Ustedes| Empiezan    | Empezaron   | Empezaban   | Empezarían     | Empezarán    |
| **Cortar (to cut)**   | Yo                 | Corto       | Corté       | Cortaba     | Cortaría       | Cortaré      |
|                       | ...                | ...         | ...         | ...         | ...            | ...          |

Repeat this format for all the verbs provided in the list. Ensure the table is clear, consistent, and visually organized.
"""

In [42]:
# result_conjugation = chat_with_ollama(user_prompt)
result_conjugation = chat_with_gpt(user_prompt)

In [43]:
display(Markdown(result_conjugation))

Here is the conjugation table for the provided Spanish verbs following all the specified guidelines:

| Verb *(English)*      | Pronoun            | Present      | Preterite     | Imperfect     | Conditional     | Future        |
|-----------------------|--------------------|--------------|---------------|---------------|------------------|---------------|
| **Poner (to put)**    | Yo                 | Pongo        | Puse          | Ponía         | Pondría          | Pondré        |
|                       | Tú                 | Pones        | Pusiste       | Ponías        | Pondrías         | Pondrás       |
|                       | Él/Ella/Usted      | Pone         | Puso          | Ponía         | Pondría          | Pondrá        |
|                       | Nosotros           | Ponemos      | Pusimos       | Poníamos      | Pondríamos       | Pondremos     |
|                       | Ellos/Ellas/Ustedes| Ponen        | Pusieron      | Ponían        | Pondrían         | Pondrán       |
| **Nadar (to swim)**   | Yo                 | Nado         | Nadé          | Nadaba        | Nadaría          | Nadaré        |
|                       | Tú                 | Nadas        | Nadaste       | Nadabas       | Nadarías         | Nadarás       |
|                       | Él/Ella/Usted      | Nada         | Nadó          | Nadaba        | Nadaría          | Nadará        |
|                       | Nosotros           | Nadamos      | Nadamos       | Nadábamos     | Nadaríamos       | Nadaremos     |
|                       | Ellos/Ellas/Ustedes| Nadan        | Nadar        | Nadaban       | Nadarían         | Nadarán       |
| **Besar (to kiss)**   | Yo                 | Beso         | Besé          | Besaba        | Besaría          | Besaré        |
|                       | Tú                 | Besas        | Besaste       | Besabas       | Besarías         | Besarás       |
|                       | Él/Ella/Usted      | Besa         | Besó          | Besaba        | Besaría          | Besará        |
|                       | Nosotros           | Besamos      | Besamos       | Besábamos     | Besaríamos       | Besaremos     |
|                       | Ellos/Ellas/Ustedes| Besan        | Besaron       | Besaban       | Besarían         | Besarán       |
| **Conseguir (to find/get)** | Yo           | Consigo      | Conseguí      | Conseguía     | Conseguiría      | Conseguiré    |
|                       | Tú                 | Consigues    | Conseguiste   | Conseguías    | Conseguirías     | Conseguirás   |
|                       | Él/Ella/Usted      | Consigue     | Consiguió     | Conseguía     | Conseguiría      | Conseguirá    |
|                       | Nosotros           | Conseguimos  | Conseguimos   | Conseguíamos  | Conseguiríamos   | Conseguiremos |
|                       | Ellos/Ellas/Ustedes| Consiguen    | Consiguieron  | Conseguían    | Conseguirían     | Conseguirán   |
| **Recomendar (to recommend)** | Yo        | Recomiendo   | Recomendé     | Recomendaba   | Recomendaría     | Recomendaré   |
|                       | Tú                 | Recomiendas  | Recomendaste  | Recomendabas  | Recomendarías    | Recomendarás  |
|                       | Él/Ella/Usted      | Recomienda   | Recomendó     | Recomendaba   | Recomendaría     | Recomendará   |
|                       | Nosotros           | Recomendamos  | Recomendamos  | Recomendábamos | Recomendaríamos  | Recomendaríamos |
|                       | Ellos/Ellas/Ustedes| Recomiendan  | Recomendaron  | Recomendaban  | Recomendarían    | Recomendarán  |
| **Salir (to go out)** | Yo                 | Salgo        | Salí          | Salía         | Saldría          | Saldré        |
|                       | Tú                 | Sales        | Saliste       | Salías        | Saldrías         | Saldrás       |
|                       | Él/Ella/Usted      | Sale         | Salió         | Salía         | Saldría          | Saldrá        |
|                       | Nosotros           | Salimos      | Salimos       | Salíamos      | Saldríamos       | Saliremos     |
|                       | Ellos/Ellas/Ustedes| Salen        | Salieron      | Salían        | Saldrían         | Saldrán       |
| **Escoger (to choose)** | Yo               | Escojo       | Escogí        | Escogía       | Escogería        | Escogeré      |
|                       | Tú                 | Escoges      | Escogiste     | Escogías      | Escogerías       | Escogerás     |
|                       | Él/Ella/Usted      | Escoge       | Escogió       | Escogía       | Escogería        | Escogerá      |
|                       | Nosotros           | Escogemos    | Escogimos     | Escogíamos    | Escogeríamos     | Escogeremos   |
|                       | Ellos/Ellas/Ustedes| Escogen      | Escogieron    | Escogían      | Escogerían       | Escogerán     |
| **Manejar (to drive)** | Yo                | Manejo       | Manejó        | Manejaba      | Manejaría        | Manejaré      |
|                       | Tú                 | Manejas      | Manejaste     | Manejabas     | Manejarías       | Manejarás     |
|                       | Él/Ella/Usted      | Maneja       | Manejó        | Manejaba      | Manejaría        | Manejará      |
|                       | Nosotros           | Manejamos    | Manejamos     | Manejábamos   | Manejaríamos     | Manejaremos   |
|                       | Ellos/Ellas/Ustedes| Manejan      | Manejan       | Manejaban     | Manejarían       | Manejarán     |
| **Ahorrar (to save)** | Yo                | Ahorro       | Ahorré        | Ahorraba      | Ahorraría        | Ahorraré      |
|                       | Tú                 | Ahorras      | Ahorraste     | Ahorrabas     | Ahorrarías       | Ahorrarás     |
|                       | Él/Ella/Usted      | Ahorra       | Ahorró        | Ahorraba      | Ahorraría        | Ahorrará      |
|                       | Nosotros           | Ahorramos    | Ahorramos     | Ahorrábamos   | Ahorraríamos     | Ahorraremos   |
|                       | Ellos/Ellas/Ustedes| Ahorran      | Ahorraron     | Ahorraban     | Ahorrarían       | Ahorrarán     |
| **Pintar (to paint)** | Yo                | Pinto        | Pinté         | Pintaba       | Pintaría         | Pintaré       |
|                       | Tú                 | Pintas       | Pintaste      | Pintabas      | Pintarías        | Pintarás      |
|                       | Él/Ella/Usted      | Pinta        | Pintó         | Pintaba       | Pintaría         | Pintará       |
|                       | Nosotros           | Pintamos     | Pintamos      | Pintábamos    | Pintaríamos      | Pintaremos    |
|                       | Ellos/Ellas/Ustedes| Pintan       | PIntaron      | Pintaban      | Pintarían        | Pintarán      |
| **Esperar (to wait/expect)** | Yo        | Espero       | Esperé        | Esperaba      | Esperaría        | Esperaré      |
|                       | Tú                 | Esperas      | Esperaste     | Esperabas     | Esperarías       | Esperarás     |
|                       | Él/Ella/Usted      | Espera       | Esperó        | Esperaba      | Esperaría        | Esperará      |
|                       | Nosotros           | Esperamos    | Esperamos     | Esperábamos   | Esperaríamos     | Esperaremos   |
|                       | Ellos/Ellas/Ustedes| Esperan      | Esperaron     | Esperaban     | Esperarían       | Esperarán     |
| **Doler (to hurt)**   | Yo                 | Duele        | Dolió         | Dolía         | Dolería          | Doleré        |
|                       | Tú                 | Duele        | Dolió         | Dolías        | Dolerías         | Dolerás       |
|                       | Él/Ella/Usted      | Duele        | Dolió         | Dolía         | Dolería          | Dolerá        |
|                       | Nosotros           | Duele        | Dolió         | Dolíamos      | Doleríamos       | Doleremos     |
|                       | Ellos/Ellas/Ustedes| Duele        | Dolió         | Dolían        | Dolerían         | Dolerán       |
| **Sentar(se) (to sit)** | Yo             | Me siento     | Me senté      | Me sentaba    | Me sentaría      | Me sentaré    |
|                       | Tú                 | Te sientas   | Te sentaste   | Te sentabas   | Te sentarías     | Te sentarás   |
|                       | Él/Ella/Usted      | Se sienta    | Se sentó      | Se sentaba    | Se sentaría      | Se sentará    |
|                       | Nosotros           | Nos sentamos  | Nos sentamos  | Nos sentábamos | Nos sentaríamos  | Nos sentaremos |
|                       | Ellos/Ellas/Ustedes| Se sientan   | Se sentaron   | Se sentaban   | Se sentarían     | Se sentarán   |
| **Ofrecer (to offer)**| Yo                 | Ofrezco      | Ofrecí        | Ofrecía       | Ofrecería        | Ofreceré      |
|                       | Tú                 | Ofreces      | Ofreciste     | Ofrecías      | Ofrecerías       | Ofrecerás     |
|                       | Él/Ella/Usted      | Ofrece       | Ofreció       | Ofrecía       | Ofrecería        | Ofrecerá      |
|                       | Nosotros           | Ofrecemos    | Ofrecimos     | Ofrecíamos    | Ofreceríamos     | Ofreceremos   |
|                       | Ellos/Ellas/Ustedes| Ofrecen      | Ofrecieron    | Ofrecían      | Ofrecerían       | Ofrecerán     |
| **Parar (to stop)**   | Yo                 | Paro         | Paré          | Paraba        | Pararía          | Pararé        |
|                       | Tú                 | Paras        | Paraste       | Parabas       | Pararías         | Pararás       |
|                       | Él/Ella/Usted      | Para         | Paró          | Paraba        | Pararía          | Parará        |
|                       | Nosotros           | Paramostros   | Paramostros   | Parábamos     | Pararíamos       | Pararemos     |
|                       | Ellos/Ellas/Ustedes| Paran        | Pararon       | Paraban       | Pararían         | Pararán       |
| **Perder (to lose)**  | Yo                 | Pierdo       | Perdí         | Perdía        | Perdería         | Perderé       |
|                       | Tú                 | Pierdes      | Perdiste      | Perdías       | Perderías        | Perderás      |
|                       | Él/Ella/Usted      | Pierde       | Perdió        | Perdía        | Perdería         | Perderá       |
|                       | Nosotros           | Perdemos     | Perdimos      | Perdíamos     | Perderíamos      | Perderemos    |
|                       | Ellos/Ellas/Ustedes| Pierden      | Perdieron     | Perdían       | Perderían        | Perderán      |
| **Regalar (to gift)**  | Yo                | Regalo       | Regalé        | Regalaba      | Regalaría        | Regalaré      |
|                       | Tú                 | Regalas      | Regalaste     | Regalabas     | Regalarías       | Regalarás     |
|                       | Él/Ella/Usted      | Regala       | Regaló        | Regalaba      | Regalaría        | Regalará      |
|                       | Nosotros           | Regalamos    | Regalamos     | Regalábamos   | Regalaríamos     | Regaleremos   |
|                       | Ellos/Ellas/Ustedes| Regalan      | Regalaron     | Regalaban     | Regalarían       | Regalarán     |
| **Recibir (to receive)** | Yo             | Recibo       | Recibí        | Recibía       | Recibiría        | Recibiré      |
|                       | Tú                 | Recibes      | Recibiste     | Recibías      | Recibirías       | Recibirás     |
|                       | Él/Ella/Usted      | Recibe       | Recibió       | Recibía       | Recibiría        | Recibirá      |
|                       | Nosotros           | Recibimos    | Recibimos     | Recibíamos    | Recibiríamos     | Recibiremos   |
|                       | Ellos/Ellas/Ustedes| Reciben      | Recibieron    | Recibían      | Recibirían       | Recibirán     |
| **Contar (to count)** | Yo                 | Cuento       | Conté         | Contaba       | Contaría         | Contaré       |
|                       | Tú                 | Cuentas      | Contaste      | Contabas      | Contarías        | Contarás      |
|                       | Él/Ella/Usted      | Cuenta       | Contó         | Contaba       | Contaría         | Contará       |
|                       | Nosotros           | Contamos     | Contamos      | Contábamos    | Contaríamos      | Contaremos    |
|                       | Ellos/Ellas/Ustedes| Cuentan      | Contaron      | Contaban      | Contarían        | Contarán      |
| **Saltar (to jump)**  | Yo                 | Salto        | Salté         | Saltaba       | Saltaría         | Saltaré       |
|                       | Tú                 | Saltas       | Saltaste      | Saltabas      | Saltarías        | Saltarás      |
|                       | Él/Ella/Usted      | Salta        | Saltó         | Saltaba       | Saltaría         | Saltará       |
|                       | Nosotros           | Saltamos     | Saltamos      | Saltábamos    | Saltaríamos      | Saltaremos    |
|                       | Ellos/Ellas/Ustedes| Saltan       | Saltaron      | Saltaban      | Saltarían        | Saltarán      |
| **Actuar (to act)**   | Yo                 | Actúo        | Actué         | Actuaba       | Actuaría         | Actuaré       |
|                       | Tú                 | Actúas      | Actuaste      | Actuabas      | Actuarías        | Actuarás      |
|                       | Él/Ella/Usted      | Actúa       | Actuó         | Actuaba       | Actuaría         | Actuará       |
|                       | Nosotros           | Actuamos    | Actuamos      | Actuábamos    | Actuaríamos      | Actuaremos    |
|                       | Ellos/Ellas/Ustedes| Actúan       | Actuaron      | Actuaban      | Actuarían        | Actuarán      |
| **Disfrutar (to enjoy)** | Yo             | Disfruto     | Disfruté      | Disfrutaba    | Disfrutaría      | Disfrutaré    |
|                       | Tú                 | Disfrutas    | Disfrutaste   | Disfrutabas   | Disfrutarías     | Disfrutarás   |
|                       | Él/Ella/Usted      | Disfruta     | Disfrutó      | Disfrutaba    | Disfrutaría      | Disfrutará    |
|                       | Nosotros           | Disfrutamos  | Disfrutamos   | Disfrutábamos | Disfrutaríamos   | Disfrutaremos  |
|                       | Ellos/Ellas/Ustedes| Disfrutan    | Disfrutaron   | Disfrutaban   | Disfrutarían     | Disfrutarán   |
| **Empezar (to start)**| Yo                 | Empiezo      | Empecé        | Empezaba      | Empezaría        | Empezaré      |
|                       | Tú                 | Empiezas     | Empezaste     | Empezabas     | Empezarías       | Empezarás     |
|                       | Él/Ella/Usted      | Empieza      | Empezó        | Empezaba      | Empezaría        | Empezará      |
|                       | Nosotros           | Empezamos    | Empezamos     | Empezábamos    | Empezaríamos   | Empezaremos   |
|                       | Ellos/Ellas/Ustedes| Empiezan     | Empezaron     | Empezaban     | Empezarían       | Empezarán     |
| **Cortar (to cut)**   | Yo                 | Corto        | Corté         | Cortaba       | Cortaría         | Cortaré       |
|                       | Tú                 | Cortas       | Cortaste      | Cortabas      | Cortarías        | Cortarás      |
|                       | Él/Ella/Usted      | Corta        | Cortó         | Cortaba       | Cortaría         | Cortará       |
|                       | Nosotros           | Cortamos     | Cortamos      | Cortábamos    | Cortaríamos      | Cortaremos    |
|                       | Ellos/Ellas/Ustedes| Cortan       | Cortaron      | Cortaban      | Cortarían        | Cortarán      |
| **Descansar (to rest)**| Yo                | Descanzo     | Descansé      | Descansaba    | Descansaría      | Descansaré    |
|                       | Tú                 | Descansas    | Descansaste   | Descansabas   | Descansarías     | Descansarás   |
|                       | Él/Ella/Usted      | Descansa     | Descansó      | Descansaba    | Descansaría      | Descansará    |
|                       | Nosotros           | Descansamos   | Descansamos   | Descansábamos | Descansaríamos   | Descansaremos  |
|                       | Ellos/Ellas/Ustedes| Descansan    | Descansaron   | Descansaban   | Descansarían     | Descansarán   |
| **Elegir (to choose)** | Yo                | Elijo        | Elegí         | Elegía        | Elegiría         | Elegiré       |
|                       | Tú                 | Eliges       | Elegiste      | Elegías       | Elegirías        | Elegirás      |
|                       | Él/Ella/Usted      | Elige        | Eligió        | Elegía        | Elegiría         | Elegirá       |
|                       | Nosotros           | Elegimos     | Elegimos      | Elegíamos     | Elegiríamos      | Elegiremos    |
|                       | Ellos/Ellas/Ustedes| Eligen       | Eligieron     | Elegían       | Elegirían        | Elegirán      |
| **Llorar (to cry)**   | Yo                 | Lloro        | Lloré         | Lloraba       | Lloraría         | Lloraré       |
|                       | Tú                 | Lloras       | Lloraste      | Llorabas      | Llorarías        | Llorarás      |
|                       | Él/Ella/Usted      | Llora        | Lloró         | Lloraba       | Lloraría         | Llorará       |
|                       | Nosotros           | Lloramos     | Lloramos      | Llorábamos    | Lloraríamos      | Lloraremos    |
|                       | Ellos/Ellas/Ustedes| Lloran       | Lloraron      | Lloraban      | Llorarían        | Llorarán      |
| **Caer (to fall)**    | Yo                 | Caigo        | Caí           | Caía          | Caería           | Caeré         |
|                       | Tú                 | Caes        | Caíste        | Caías         | Caerías          | Caerás        |
|                       | Él/Ella/Usted      | Cae          | Cayó          | Caía          | Caería           | Caerá         |
|                       | Nosotros           | Caemos       | Caímos        | Caíamos       | Caeríamos        | Caeremos      |
|                       | Ellos/Ellas/Ustedes| Caen         | Cayeron       | Caían         | Caerían          | Caerán        |
| **Invitar (to invite)**| Yo                | Invito      | Invit           | Invitaba      | Invitaría        | Invitaré      |
|                       | Tú                 | Invitas      | Invitaste      | Invitabas     | Invitarías       | Invitarás     |
|                       | Él/Ella/Usted      | Invita       | Invitó         | Invitaba      | Invitaría        | Invitará      |
|                       | Nosotros           | Invitamos    | Invitamos      | Invitábamos   | Invitaríamos     | Invitaremos    |
|                       | Ellos/Ellas/Ustedes| Invitan      | Invitaron      | Invitaban     | Invitarían       | Invitarán     |
| **Sonreír (to smile)**| Yo                 | Sonrío      | Sonreí         | Sonreía       | Sonreiría       | Sonreiré      |
|                       | Tú                 | Sonríes      | Sonreíste     | Sonreías      | Sonreirías       | Sonreirás     |
|                       | Él/Ella/Usted      | Sonríe       | Sonrió         | Sonreía       | Sonreiría       | Sonreirá      |
|                       | Nosotros           | Sonreímos    | Sonreímos      | Sonreíamos    | Sonreiríamos     | Sonreiremos   |
|                       | Ellos/Ellas/Ustedes| Sonríen      | Sonrieron      | Sonreían      | Sonreirían       | Sonreirán     |
| **Traer (to bring)**  | Yo                 | Traigo       | Traje          | Traía         | Traería          | Traeré        |
|                       | Tú                 | Traes        | Trajiste       | Traías        | Traerías         | Traerás       |
|                       | Él/Ella/Usted      | Trae         | Trajo          | Traía         | Traería          | Traerá        |
|                       | Nosotros           | Traemos      | Trajimos       | Traíamos      | Traeríamos       | Traeremos     |
|                       | Ellos/Ellas/Ustedes| Traen        | Trajeron       | Traían        | Traerían         | Traerán       |
| **Visitar (to visit)**| Yo                 | Visito       | Visité        | Visitaba      | Visitaría        | Visitaré      |
|                       | Tú                 | Visitas      | Visitaste     | Visitabas     | Visitarías       | Visitarás     |
|                       | Él/Ella/Usted      | Visita       | Visitó        | Visitaba      | Visitaría        | Visitará      |
|                       | Nosotros           | Visitamos    | Visitamos     | Visitábamos   | Visitaríamos     | Visitaremos   |
|                       | Ellos/Ellas/Ustedes| Visitan      | Visitaron     | Visitaban     | Visitarían       | Visitarán     |
| **Volar (to fly)**    | Yo                 | Vuelo        | Volé          | Volaba        | Volaría          | Volaré        |
|                       | Tú                 | Vuelas       | Volaste       | Volabas       | Volarías         | Volarás       |
|                       | Él/Ella/Usted      | Vuela        | Voló          | Volaba        | Volaría          | Volará        |
|                       | Nosotros           | Volamos      | Volamos       | Volábamos     | Volaríamos       | Volaremos     |
|                       | Ellos/Ellas/Ustedes| Vuelan       | Volaron       | Volaban       | Volarían         | Volarán       |
| **Abrir (to open)**   | Yo                 | Abro         | Abrí          | Abría         | Abriría          | Abriré        |
|                       | Tú                 | Abres        | Abriste       | Abrías        | Abrirías         | Abrirás       |
|                       | Él/Ella/Usted      | Abre         | Abrió         | Abría         | Abriría          | Abrirá        |
|                       | Nosotros           | Abrimos      | Abrimos       | Abríamos      | Abriríamos       | Abriremos     |
|                       | Ellos/Ellas/Ustedes| Abren        | Abrieron      | Abrían        | Abrirían         | Abrirán       |
| **Cerrar (to close)** | Yo                 | Cierro       | Cerré         | Cerraba       | Cerraré          | Cerraré       |
|                       | Tú                 | Cierras      | Cerraste      | Cerrabas      | Cerrarías        | Cerrarás      |
|                       | Él/Ella/Usted      | Cierra       | Cerró         | Cerraba       | Cerrarías        | Cerrará       |
|                       | Nosotros           | Cerramos     | Cerramos      | Cerrábamos    | Cerraremos       | Cerraremos    |
|                       | Ellos/Ellas/Ustedes| Cierran      | Cerraron      | Cerraban      | Cerrarían        | Cerrarán      |
| **Bailar (to dance)** | Yo                 | Bailo        | Bailé         | Bailaba       | Bailaría         | Bailaré       |
|                       | Tú                 | Bailas       | Bailaste      | Bailabas      | Bailarías        | Bailarás      |
|                       | Él/Ella/Usted      | Baila        | Bailó         | Bailaba       | Bailaría         | Bailará       |
|                       | Nosotros           | Bailamos     | Bailamos      | Bailábamos    | Bailaríamos      | Bailaremos    |
|                       | Ellos/Ellas/Ustedes| Bailan       | Bailaron      | Bailaban      | Bailarían        | Bailarán      |
| **Crear (to create)** | Yo                 | Creo         | Creé          | Creaba        | Crearé           | Crearé        |
|                       | Tú                 | Creas        | Creaste       | Creabas       | Crearías         | Crearás       |
|                       | Él/Ella/Usted      | Crea         | Creó          | Creaba        | Crearía          | Creará        |
|                       | Nosotros           | Creamos      | Creamos       | Creamos       | Crearemos        | Crearemos     |
|                       | Ellos/Ellas/Ustedes| Crean        | Crearon       | Creaban       | Crearían         | Crearán       |
| **Llamar (to call)**  | Yo                 | Llamo        | Llamé         | Llamaba       | Llamaría         | Llamaré       |
|                       | Tú                 | Llamas       | Llamaste      | Llamabas      | Llamarías        | Llamarás      |
|                       | Él/Ella/Usted      | Llama        | Llamó         | Llamaba       | Llamaría         | Llamará       |
|                       | Nosotros           | Llamamos     | Llamamos      | Llamábamos    | Llamaríamos      | Llamaremos     |
|                       | Ellos/Ellas/Ustedes| Llaman       | Llamaron      | Llamaban      | Llamarían        | Llamarán      |
| **Oír (to hear)**     | Yo                 | Oigo         | Oí            | Oía           | Oiría            | Oiré          |
|                       | Tú                 | Oyes         | Oíste         | Oías          | Oirías           | Oirás         |
|                       | Él/Ella/Usted      | Oye          | Oyó           | Oía           | Oiría            | Oirá          |
|                       | Nosotros           | Oímos        | Oímos         | Oíamos        | Oíríamos         | Oiremos       |
|                       | Ellos/Ellas/Ustedes| Oyen         | Oyeron        | Oían          | Oirían           | Oirán         |
| **Olvidar (to forget)** | Yo               | Olvido       | Olvidé        | Olvidaba      | Olvidaría        | Olvidaré      |
|                       | Tú                 | Olvidas      | Olvidaste     | Olvidabas     | Olvidarías       | Olvidarás     |
|                       | Él/Ella/Usted      | Olvida       | Olvidó        | Olvidaba      | Olvidaría        | Olvidará      |
|                       | Nosotros           | Olvidamos    | Olvidamos     | Olvidábamos   | Olvidaríamos     | Olvidaremos    |
|                       | Ellos/Ellas/Ustedes| Olvidan      | Olvidaron     | Olvidaban     | Olvidarían       | Olvidarán     |
| **Comprar (to buy)**  | Yo                 | Compro       | Compré        | Compraba      | Compraré         | Compraré      |
|                       | Tú                 | Compras      | Compraste     | Comprabas     | Comprarías       | Comprarás     |
|                       | Él/Ella/Usted      | Compra       | Compró        | Compraba      | Compraría        | Comprará      |
|                       | Nosotros           | Compramos    | Compramos     | Comprábamos   | Compraríamos     | Compraremos   |
|                       | Ellos/Ellas/Ustedes| Compran      | Compraron     | Compraban     | Comprarán        | Comprarán     |
| **Vender (to sell)**  | Yo                 | Vendo        | Vendí         | Vendía        | Vendería         | Venderé       |
|                       | Tú                 | Vendes       | Vendiste      | Vendías       | Venderías        | Venderás      |
|                       | Él/Ella/Usted      | Vende        | Vendió        | Vendía        | Vendería         | Venderá       |
|                       | Nosotros           | Vendemos     | Vendimos      | Vendíamos     | Venderíamos      | Venderemos    |
|                       | Ellos/Ellas/Ustedes| Venden       | Vendieron     | Vendían       | Venderían        | Venderán      |
| **Enseñar (to teach)**| Yo                 | Enseño       | Enseñé        | Enseñaba      | Enseñaría        | Enseñaré      |
|                       | Tú                 | Enseñas      | Enseñaste     | Enseñabas     | Enseñarías       | Enseñarás     |
|                       | Él/Ella/Usted      | Enseña       | Enseñó        | Enseñaba      | Enseñaría        | Enseñará      |
|                       | Nosotros           | Enseñamos    | Enseñamos     | Enseñábamos   | Enseñaríamos     | Enseñaremos   |
|                       | Ellos/Ellas/Ustedes| Enseñan      | Enseñaron     | Enseñaban     | Enseñarían       | Enseñarán     |
| **Describir (to describe)**| Yo          | Describo      | Describí      | Describía     | Describiría      | Describiré    |
|                       | Tú                 | Describes    | Describiste   | Describías    | Describirías     | Describirás   |
|                       | Él/Ella/Usted      | Describe      | Describió     | Describía     | Describiría      | Describirá    |
|                       | Nosotros           | Describimos  | Describimos   | Describíamos  | Describiríamos   | Describiremos  |
|                       | Ellos/Ellas/Ustedes| Describen    | Describieron  | Describían    | Describirían     | Describirán   |
| **Mirar (to look at)** | Yo                | Miro         | Miré          | Miraba        | Miraría          | Miraré        |
|                       | Tú                 | Miras        | Miraste       | Mirabas       | Mirarías         | Mirarás       |
|                       | Él/Ella/Usted      | Mira         | Miró          | Miraba        | Miraría          | Mirará        |
|                       | Nosotros           | Miramos      | Miramos       | Mirábamos     | Miraríamos       | Miraremos     |
|                       | Ellos/Ellas/Ustedes| Miran        | Miraron       | Miraban       | Mirarían         | Mirarán       |
| **Quedarse (to stay)**| Yo                 | Me quedo     | Me quedé      | Me quedaba    | Me quedaría      | Me quedaré    |
|                       | Tú                 | Te quedas    | Te quedaste   | Te quedabas   | Te quedarías     | Te quedarás   |
|                       | Él/Ella/Usted      | Se queda     | Se quedó      | Se quedaba    | Se quedaría      | Se quedará    |
|                       | Nosotros           | Nos quedamos  | Nos quedamos  | Nos quedábamos | Nos quedaríamos  | Nos quedaremos |
|                       | Ellos/Ellas/Ustedes| Se quedan    | Se quedaron    | Se quedaban   | Se quedarían     | Se quedarán   |


In [44]:
# result_conjugation
with open("output/combined_text.txt", "w", encoding="utf-8") as output_file:
    output_file.write(result_conjugation)

In [ ]:
import pandas as pd
import re

def save_as_csv(text, file_name="output/spanish_verbs_conjugation.csv"):

    # Extract the header and rows using regular expressions
    lines = text.splitlines()
    table_lines = [line for line in lines if "|" in line]  # Keep only table lines

    # Process the header and rows
    header = [col.strip() for col in table_lines[0].split("|")[1:-1]]  # Extract header
    data = [
        [col.strip() for col in row.split("|")[1:-1]]  # Extract each row
        for row in table_lines[2:]  # Skip separator row
    ]

    # Create a DataFrame
    df = pd.DataFrame(data, columns=header)

    # Save to CSV with UTF-8 BOM
    df.to_csv(file_name, index=False, encoding="utf-8-sig")

    print(f"CSV file saved as {file_name}' with UTF-8 BOM.")

In [51]:
save_as_csv(result_conjugation)

CSV file saved as output/spanish_verbs_conjugation.csv' with UTF-8 BOM.
